In [ ]:
import os
import shutil
from PIL import Image
import numpy as np

food_classes = ['burger', 'pizza', 'salad']
base_path = 'food_data'

if os.path.exists(base_path):
    shutil.rmtree(base_path)

for cls in food_classes:
    os.makedirs(os.path.join(base_path, 'train', cls))
    os.makedirs(os.path.join(base_path, 'val', cls))


In [ ]:
def generate_dummy_images(folder, label, color, count=20):
    for i in range(count):
        img = Image.new('RGB', (224, 224), color=color)
        img.save(os.path.join(folder, label, f'{label}_{i}.jpg'))

generate_dummy_images('food_data/train', 'burger', (255, 200, 200), 25)
generate_dummy_images('food_data/train', 'pizza', (200, 255, 200), 25)
generate_dummy_images('food_data/train', 'salad', (200, 200, 255), 25)

generate_dummy_images('food_data/val', 'burger', (255, 200, 200), 5)
generate_dummy_images('food_data/val', 'pizza', (200, 255, 200), 5)
generate_dummy_images('food_data/val', 'salad', (200, 200, 255), 5)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    'food_data/train',
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)

val_generator = datagen.flow_from_directory(
    'food_data/val',
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)


Found 75 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,907,779 (91.20 MB)

 Trainable params: 23,907,779 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4838 - loss: 7.8599 - val_accuracy: 1.0000 - val_loss: 0.4559
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 960ms/step - accuracy: 0.6618 - loss: 0.5785 - val_accuracy: 0.6667 - val_loss: 0.3116
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 871ms/step - accuracy: 0.8413 - loss: 0.3111 - val_accuracy: 1.0000 - val_loss: 0.1157
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 996ms/step - accuracy: 0.9613 - loss: 0.1213 - val_accuracy: 1.0000 - val_loss: 0.0257
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 983ms/step - accuracy: 1.0000 - loss: 0.0339 - val_accuracy: 1.0000 - val_loss: 0.0064


In [ ]:
from google.colab import files

uploaded = files.upload()

for file_name in uploaded.keys():
    img = image.load_img(file_name, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    print(f"Predicted class for {file_name}: {class_labels[predicted_class]}")


Saving download.jpeg to download.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted class for download.jpeg: burger
